# Get Papers from Mohit Bansal

In [ ]:
from typing import *
import csv
import pickle

from matplotlib import pyplot as plt
from tqdm import tqdm

import workspace
from scholar_api import *
from gpt import rate

In [ ]:
AUTHOR_ID = '143977268'

PROMPT = '''
We're writing a paper about a prompt manager that integrates an LLM into a complex system. The LLM directs the user interaction with the system. The LLM serves as the interface between the user and the system. I'm looking for related works, e.g., LLM as a controller, as a user interfaces, as a tools director, or as an assistant.

Evaluate the relevance of the following paper to my research.

<paper>
%s
</paper>

Is the above paper very relevant? Be strict and don't admit loosely-related works. Answer "Yes" or "No", using exactly one single word.
'''.strip()

In [ ]:
papers = getPapersFromAuthor(AUTHOR_ID)
len(papers), papers[0]

In [ ]:
with open('citers.pickle', 'rb') as f:
    citers = pickle.load(f)
citers_set = set([x[PAPERID] for x in citers])
papers_new = [x for x in papers if x[PAPERID] not in citers_set]
len(papers_new)

In [ ]:
rated = {}

In [ ]:
for paper in tqdm(papers_new):
    paper_id = paper[PAPERID]
    title = paper[TITLE] or 'None'
    abstract = paper[ABSTRACT] or 'None'
    if paper_id in rated:
        continue
    print()
    print(title)
    score = rate(PROMPT % (title + '\nAbstract: ' + abstract).strip())
    relevance = format(score, '.3%')
    print(f'{relevance = }')
    rated[paper_id] = (title, abstract, score, relevance)


In [ ]:
len(rated)

In [ ]:
papers = [*rated.items()]
papers.sort(key=lambda x: x[1][2], reverse=True)

In [ ]:
plt.hist([x[1][2] for x in papers], bins=10)

In [ ]:
with open('results.csv', 'w', encoding='utf-8', newline='') as fcsv:
    c = csv.writer(fcsv)
    for paper_id, (title, abstract, score, relevance) in papers:
        print(relevance)
        print(title)
        print(abstract)
        print()

        c.writerow((relevance, title, abstract, paper_id))
